# AWS Lambdaを MCPify する
## Bedrock AgentCore Gatewayを使って、AWS Lambdaの関数をセキュアなMCPツールに変換する

## 概要
Bedrock AgentCore Gatewayは、お客様が既存のAWS Lambdaの関数を、インフラやホスティングを管理する必要なく、完全に管理されたMCPサーバーに変換する方法を提供します。Gatewayは、これらすべてのツールに対して統一されたModel Context Protocol (MCP)インターフェイスを提供します。Gatewayは、双方向の認証モデルを採用しており、インバウンド要求とターゲットリソースへのアウトバウンド接続の両方に対してセキュアなアクセス制御を保証します。このフレームワークは、2つの主要コンポーネントで構成されています。Inbound Authは、ゲートウェイターゲットにアクセスしようとするユーザーを検証および承認します。Outbound Authは、ゲートウェイが認証されたユーザーに代わって、バックエンドリソースに安全に接続できるようにします。Gatewayは、アウトバウンド認証のためにIAMロールを使用して、AWS Lambdaの関数への呼び出しを承認します。

![How does it work](images/lambda-iam-gateway.png)

### チュートリアルの詳細

| 情報                 | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルの種類 | インタラクティブ                                          |
| AgentCoreコンポーネント | AgentCore Gateway、AgentCore Identity                    |
| Agenticフレームワーク | Strands Agents                                           |
| ゲートウェイターゲットの種類 | AWS Lambda                                         |
| インバウンド認証IdP  | Amazon Cognito                                            |
| アウトバウンド認証   | AWS IAM                                                   |
| LLMモデル            | Anthropic Claude Sonnet 3.7、Amazon Nova Pro              |
| チュートリアルコンポーネント | AgentCore Gatewayの作成とAgentCore Gatewayの呼び出し     |
| チュートリアルの分野 | 分野横断                                                  |
| 例の複雑さ           | 簡単                                                      |
| 使用したSDK          | boto3                                                     |

チュートリアルの最初の部分では、いくつかのAmazonCore Gatewayターゲットを作成します。

### チュートリアルのアーキテクチャ
このチュートリアルでは、AWS Lambdaの関数で定義された操作をMCPツールに変換し、Bedrock AgentCore Gatewayでホストします。
デモンストレーション目的で、Amazon Bedrockモデルを使用するStrands Agentを使用します。
私たちの例では、get_orderとupdate_orderの2つのツールを持つ非常に単純なエージェントを使用します。

## 前提条件

このチュートリアルを実行するには、以下が必要です:
* Jupyter notebook (Python kernel)
* uv
* AWS 認証情報
* Amazon Cognito

## AgentCore Gateway への着信リクエストの認証設定

AgentCore Gateway は、着信認証と発信認証を通じて安全な接続を提供します。着信認証では、AgentCore Gateway は呼び出し時に渡された OAuth トークンを分析し、ゲートウェイ内のツールへのアクセスを許可または拒否します。ツールが外部リソースへのアクセスを必要とする場合、AgentCore Gateway は API キー、IAM、または OAuth トークンを使って発信認証を行い、外部リソースへのアクセスを許可または拒否できます。

着信認証フローでは、エージェントまたは MCP クライアントが、ユーザーの IdP から生成された OAuth アクセストークンを付加して、AgentCore Gateway 内の MCP ツールを呼び出します。AgentCore Gateway は OAuth アクセストークンを検証し、着信認証を実行します。

AgentCore Gateway 内で実行されるツールが外部リソースにアクセスする必要がある場合、OAuth はゲートウェイターゲットのリソース認証情報プロバイダを使って、下流のリソースの認証情報を取得します。AgentCore Gateway は認証情報を呼び出し元に渡し、下流の API へのアクセスを許可します。

In [ ]:
# Make sure you download the latest botocore and boto3 libraries.

日本語訳:

最新の botocore と boto3 ライブラリをダウンロードしていることを確認してください。
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook

Amazon SageMaker ノートブックを使用していない場合は、以下の手順で AWS 認証情報を設定します。
import os
os.environ['AWS_ACCESS_KEY_ID'] = '' # set the access key

アクセスキーを設定します。

access_key = "abc123"  # アクセスキーは変更しないでください
secret_key = "xYzPqr"  # 秘密キーは変更しないでください

if access_key and secret_key:
    print("Access key and secret key are set.")
else:
    print("Error: Access key or secret key is missing.")

# 関数 get_data() を定義します
def get_data(url):
    # url から データを取得する処理...
    return data

# メイン処理
url = "https://example.com/api/data"
data = get_data(url)
print(data)
os.environ['AWS_SECRET_ACCESS_KEY'] = '' # set the secret 

secret = "my_secret_password"

日本語訳:

# 秘密鍵を設定

secret = "my_secret_password"
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1' # set the AWS region

AWS リージョンを設定します。

日本語訳:

# AWS リージョンを設定する

In [ ]:
import os
import sys

# Get the directory of the current script

現在のスクリプトのディレクトリを取得します。

script_dir = os.path.dirname(os.path.abspath(__file__))

日本語訳:

# 現在のスクリプトのディレクトリを取得します

script_dir = os.path.dirname(os.path.abspath(__file__))
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

__file__ が定義されていない場合の代替手段 (例: Jupyter)

# Navigate to the directory containing utils.py (one level up)

utils.py を含むディレクトリに移動します (1 レベル上)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]

日本語訳:

# sys.pathに追加する

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入する
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語はそのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]
sys.path.insert(0, utils_dir)

# Now you can import utils

日本語訳:
# 今から utils をインポートできます
import utils

In [ ]:
#### MCP ツールに変換したい AWS Lambda 関数のサンプルを作成する
lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")

if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("Lambda function created with ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda function creation failed with message: ", lambda_resp['lambda_function_arn'])

In [ ]:
#### Gateway が引き受ける IAM ロールを作成する
import utils
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Amazon Cognito プールを作成して Gateway への受信認証を行う

resource "aws_cognito_user_pool" "pool" {
  name = "gateway-user-pool"
}

resource "aws_cognito_user_pool_client" "client" {
  name                                 = "gateway-user-pool-client"
  user_pool_id                         = aws_cognito_user_pool.pool.id
  generate_secret                      = false
  allowed_oauth_flows_user_pool_client = true
  allowed_oauth_flows                  = ["code", "implicit"]
  callback_urls                        = ["https://example.com/oauth2/idpresponse"]
  logout_urls                          = ["https://example.com"]
}

resource "aws_apigatewayv2_authorizer" "cognito" {
  api_id           = aws_apigatewayv2_api.gateway.id
  authorizer_type  = "JWT"
  identity_sources = ["$request.header.Authorization"]
  name             = "cognito-authorizer"

  jwt_configuration {
    audience = [aws_cognito_user_pool_client.client.id]
    issuer   = "https://${aws_cognito_user_pool.pool.endpoint}"
  }
}

In [ ]:
# Creating Cognito User Pool

日本語訳:

# Cognito ユーザープールの作成

Amazon Cognito は、Web および モバイルアプリケーションのための認証、承認、およびユーザー管理を提供する AWS のサービスです。Cognito ユーザープール は、ユーザーディレクトリを提供し、ユーザーの登録、認証、アカウント復旧などの操作を処理します。

ユーザープールを作成するには、AWS マネジメントコンソールで以下の手順を実行します。

1. Cognito コンソールを開きます。
2. 「ユーザープールの管理」をクリックします。
3. 「ユーザープールの作成」をクリックします。
4. プールの名前を入力します。
5. 必要に応じて、プールの編集設定を構成します (例: パスワードポリシー、MFA 設定、属性マッピングなど)。
6. 「ユーザープールの作成」をクリックして作成を完了します。

作成後、アプリケーションで認証を有効にするための アプリクライアント ID と アプリクライアントシークレット が発行されます。これらの値は、Cognito との統合で使用されます。

注: コードやコマンド、変数名、関数名などの技術的な用語は翻訳せずにそのまま残しました。
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL

discovery_url = "https://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/token"

headers = {"Metadata-Flavor": "Google"}
r = requests.get(discovery_url, headers=headers)
r.raise_for_status()
access_token = r.text

日本語訳:

# 検出 URL を取得

discovery_url = "https://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/token"

headers = {"Metadata-Flavor": "Google"}
r = requests.get(discovery_url, headers=headers)
r.raise_for_status()
access_token = r.text
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# Amazon Cognitoの認証者を使って、インバウンド認証のためのゲートウェイを作成する

日本語訳:

Amazon Cognito 認証者を使って、インバウンド認証のためのゲートウェイを作成します。

resource "aws_apigatewayv2_api" "lambda" {
  name                       = "serverless_lambda_gw"
  protocol_type              = "HTTP"
  description                = "HTTP API Gateway"
  disable_execute_api_endpoint = true
}

resource "aws_apigatewayv2_authorizer" "api_gw_authorizer" {
  api_id           = aws_apigatewayv2_api.lambda.id
  authorizer_type  = "JWT"
  identity_sources = ["$request.header.Authorization"]
  name             = "cognito-authorizer"

  jwt_configuration {
    audience = [aws_cognito_user_pool_client.client.id]
    issuer   = "https://${aws_cognito_user_pool.pool.endpoint}"
  }
}

resource "aws_apigatewayv2_stage" "lambda" {
  api_id      = aws_apigatewayv2_api.lambda.id
  auto_deploy = true
  name        = "$default"
}

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step

日本語訳:

# CMK を使用せずに Cognito 認証者で CreateGateway を作成します。前のステップで作成した Cognito ユーザープールを使用してください。
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g

日本語訳:
# Client は Cognito で設定された ClientId と一致する必要があります。例: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='TestGWforLambda',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM ロールには Gateway を作成/一覧表示/取得/削除する権限が必要です。

日本語訳:
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with AWS Lambda target type'
)
print(create_response)
# GatewayTarget作成に使用されるGatewayIDを取得する

data "aws_ec2_transit_gateway" "example" {
  transit_gateway_id = var.transit_gateway_id
}

resource "aws_ec2_transit_gateway_route_table" "example" {
  transit_gateway_id = data.aws_ec2_transit_gateway.example.id
}

resource "aws_ram_resource_share" "example" {
  name                      = "example-tgw-share"
  allow_external_principals = true
}

resource "aws_ram_principal_association" "example" {
  principal          = data.aws_caller_identity.example.account_id
  resource_share_arn = aws_ram_resource_share.example.arn
}

resource "aws_ram_resource_association" "example" {
  resource_arn       = aws_ec2_transit_gateway.example.arn
  resource_share_arn = aws_ram_resource_share.example.arn
}

data "aws_ram_resource_share_accepters" "example" {
  resource_share_arn = aws_ram_resource_share.example.arn

  depends_on = [
    aws_ram_principal_association.example,
    aws_ram_resource_association.example,
  ]
}

output "gateway_id" {
  value       = data.aws_ec2_transit_gateway.example.id
  description = "GatewayID used for GatewayTarget creation"
}
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# AWS Lambda ターゲットを作成し、MCP ツールに変換する

日本語訳:

AWS Lambda ターゲットを作成し、それを MCP ツールに変換するには、以下の手順に従ってください。

1. AWS コンソールにログインし、 Lambda サービスを選択します。
2. 「関数の作成」をクリックします。
3. 「一から作成」を選択します。
4. 関数名を入力し (例: myMcpTools)、実行ロールを選択します。
5. 「Node.js 14.x」を選択し、「関数の作成」をクリックします。
6. コードエディタで、 index.js ファイルの内容を次のコードに置き換えます:

```javascript
exports.handler = async (event) => {
    const response = {
        statusCode: 200,
        body: JSON.stringify('Hello from Lambda!'),
    };
    return response;
};
```

7. 「デプロイ」をクリックして変更を保存します。
8. 関数の詳細ページで、「テスト」をクリックします。
9. 「新しいイベントを作成」をクリックし、イベントの名前を入力します (例: myTestEvent)。
10. テストイベントの JSON を次のように設定し、「イベントを作成」をクリックします:

```json
{
  "key1": "value1",
  "key2": "value2"
}
```

11. 「テストを実行」をクリックし、関数の実行結果を確認します。

これで、AWS Lambda 関数が作成され、MCP ツールに変換されました。今後、この関数を呼び出して、さまざまなタスクを実行できます。

In [ ]:
# Replace the AWS Lambda function ARN below

以下の AWS Lambda 関数の ARN を置き換えてください。
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_resp['lambda_function_arn'], # Replace this with your AWS Lambda function ARN

日本語訳:
# あなたの AWS Lambda 関数の ARN にこれを置き換えてください。
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_tool",
                        "description": "tool to get the order",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    },                    
                    {
                        "name": "update_order_tool",
                        "description": "tool to update the orderId",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    }
                ]
            }
        }
    }
}

credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='LambdaUsingSDK'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target using SDK',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装した Bedrock AgentCore Gateway を通じて、AWS ツールとシームレスに統合されています。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

Bedrock AgentCore Gateway の中核は、基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠の Gateway です。これらの API を使用すると、MCP 準拠のクライアントまたは SDK から、利用可能なツールを安全で標準化された方法で検出およびやり取りできます。Strands エージェントが AWS サービスにアクセスする必要がある場合は、これらの MCP 標準化されたエンドポイントを使用して Gateway と通信します。

Gateway の実装は、(MCP 認証仕様)[https://modelcontextprotocol.org/specification/draft/basic/authorization] を厳密に順守しており、堅牢なセキュリティとアクセス制御が保証されています。つまり、Strands エージェントによるすべてのツール呼び出しは、セキュリティを維持しながら強力な機能を可能にする認証ステップを経ます。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを検出し、次に InvokeTools を使用して特定のアクションを実行します。Gateway は、必要なすべてのセキュリティ検証、プロトコル変換、およびサービス対話を処理するため、プロセス全体がシームレスかつ安全に行われます。

このアーキテクチャアプローチにより、MCP 仕様を実装したクライアントまたは SDK は、Gateway を介して AWS サービスとやり取りできるため、AI エージェントの統合に対して汎用性と将来性の高い解決策となります。

![Strands agent calling Gateway](images/strands-lambda-gateway.png)

In [ ]:
uv_install("mcp[cli]", "strands-agents")

# Amazon Cognitoからアクセストークンを要求する (インバウンド認証用)

日本語訳:

アクセス トークンを取得するには、 AWS.CognitoIdentityServiceProvider クライアントを使用して、 InitiateAuth 操作を呼び出します。この操作では、認証情報 (ユーザー名とパスワード) と、クライアント ID を指定する必要があります。

応答には、認証結果と、アクセス トークン、ID トークン、更新トークンが含まれています。アクセス トークンは、Amazon API Gateway の認可者で使用されます。ID トークンには、ユーザーに関する情報 (サブジェクト、発行元、有効期限など) が含まれています。更新トークンは、アクセス トークンの有効期限が切れた場合に、新しいアクセス トークンを取得するために使用されます。

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# Strands agent calling MCP tools of AWS Lambda using Bedrock AgentCore Gateway

日本語訳:

# Strands エージェントが Bedrock AgentCore Gateway を使用して AWS Lambda の MCP ツールを呼び出す

Strands エージェントは、Bedrock AgentCore Gateway を介して AWS Lambda 上の MCP (Machine Conversation Platform) ツールを呼び出します。技術的な用語である Strands、Bedrock AgentCore Gateway、AWS Lambda、MCP は翻訳せずにそのまま残しました。半角英数字の前後には半角スペースを挿入しています。

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

以下が日本語訳になります。

## ~/.aws/credentials に設定された IAM 認証情報には、Bedrock モデルへのアクセス権限が必要です
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.

日本語訳:

# ルート strands ロガーを設定します。問題をデバッグしている場合は、これを DEBUG に変更してください。
logging.getLogger("strands").setLevel(logging.INFO)

# ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler())

logs.log("Hello, World!") # 出力: Hello, World!

logs.log(f"今の時刻は {time.time()} です") # 出力: 今の時刻は <時刻の値> です

# f-strings を使って変数を簡単に埋め込むことができます
x = 42
logs.log(f"x の値は {x} です") # 出力: x の値は 42 です
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 

listTools 関数を呼び出します。
    tools = client.list_tools_sync()
    # エージェントを model と tools で作成する

日本語訳:

# エージェントを model と tools で作成する

agent = Agent(model, tools)
    agent = Agent(model=yourmodel,tools=tools) ## お好みのモデルに置き換えることができます
    print(f"Tools loaded in the agent are {agent.tool_names}")
    print(f"Tools configuration in the agent are {agent.tool_config}")
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.

サンプルのプロンプトでエージェントを呼び出します。これは MCP listTools のみを呼び出し、LLM がアクセスできるツールのリストを取得します。以下では実際にはツールを呼び出しません。
    agent("Hi , can you list all tools available to you")
    # Invoke the agent with sample prompt, invoke the tool and display the response

サンプルのプロンプトでエージェントを呼び出し、ツールを呼び出して、応答を表示します。
    agent("Check the order status for order id 123 and show me the exact response from the tool")
    # MCP ツールを明示的に呼び出します。MCP ツール名と引数は、AWS Lambda 関数または OpenAPI/Smithy API と一致する必要があります。

日本語訳:
MCP ツールを明示的に呼び出します。MCP ツールの名前と引数は、あなたの AWS Lambda 関数または OpenAPI/Smithy API と一致する必要があります。
    result = client.call_tool_sync(
    tool_use_id="get-order-id-123-call-1", # You can replace this with unique identifier.

日本語訳:

# これは一意の識別子に置き換えることができます。
    name=targetname+"___get_order_tool", # This is the tool name based on AWS Lambda target types. This will change based on the target name

日本語訳:
これは AWS Lambda のターゲットタイプに基づくツール名です。ターゲット名に応じて変更されます。
    arguments={"orderId": "123"}
    )
    # ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler())

logs.log("Hello, World!") # 出力: Hello, World!

logs.log(f"今の時刻は {time.time()} です") # 出力: 今の時刻は <時刻の値> です

# f-strings を使って変数を簡単に埋め込むことができます
x = 42
logs.log(f"x の値は {x} です") # 出力: x の値は 42 です0
    print(f"Tool Call result: {result['content'][0]['text']}")


**問題: 以下のセルを実行中に下記のエラーが発生した場合、pydantic と pydantic-core のバージョンが互換性がないことを示しています。**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```

**解決方法:**

pydantic==2.7.2 と pydantic-core 2.27.2 が互換性があることを確認する必要があります。確認後、カーネルを再起動してください。

# クリーンアップ

IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、s3 バケットなどの追加リソースも作成されます。これらは、実行した例次第で、クリーンアップの一環として手動で削除する必要があるかもしれません。

## ゲートウェイを削除する (オプション)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)